# Variational Information Bottleneck
https://arxiv.org/abs/1612.00410

In [1]:
from __future__ import print_function
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
from tensorboardX import SummaryWriter

from tqdm import tqdm

batch_size = 100
epochs = 200
seed = 1
torch.manual_seed(seed)

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

In [2]:
root = '../data'
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Lambda(lambd=lambda x: x.view(-1)*2-1)]) # [-1,1]
kwargs = {'batch_size': batch_size, 'num_workers': 1, 'pin_memory': True}

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST(root=root, train=True, transform=transform, download=True),
    shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST(root=root, train=False, transform=transform),
    shuffle=False, **kwargs)

In [3]:
from pixyz.distributions import Normal, Categorical
from pixyz.models import Model
from pixyz.losses import StochasticExpectationLoss, KullbackLeibler
from torch.optim.lr_scheduler import StepLR

In [4]:
x_dim = 784
y_dim = 10
z_dim = 256


# encoder model p(z|x)
class Encoder(Normal):
    def __init__(self):
        super().__init__(cond_var=["x"], var=["z"], name="p")

        self.fc1 = nn.Linear(x_dim, 1024)
        self.fc2 = nn.Linear(1024, 1024)
        self.fc31 = nn.Linear(1024, z_dim)
        self.fc32 = nn.Linear(1024, z_dim)

    def forward(self, x):
        h = F.relu(self.fc1(x))
        h = F.relu(self.fc2(h))
        return {"loc": self.fc31(h),
                "scale": F.softplus(self.fc32(h)-5.0)} # see apendix

    
# classifier model q(y|z)    
class Classifier(Categorical):
    def __init__(self):
        super().__init__(cond_var=["z"], var=["y"], name="q")

        self.fc1 = nn.Linear(z_dim, y_dim)

    def forward(self, z):
        return {"probs": F.softmax(self.fc1(z), dim=1)}
    
    
# prior model r(z)
loc = torch.tensor(0.).to(device)
scale = torch.tensor(1.).to(device)
prior = Normal(loc=loc, scale=scale, var=["z"], dim=z_dim, name="r")

In [5]:
p = Encoder()
q = Classifier()

p.to(device)
q.to(device)

print(p)
print(q)

Distribution:
  p(z|x) (Normal)
Network architecture:
  Encoder(
    (fc1): Linear(in_features=784, out_features=1024, bias=True)
    (fc2): Linear(in_features=1024, out_features=1024, bias=True)
    (fc31): Linear(in_features=1024, out_features=256, bias=True)
    (fc32): Linear(in_features=1024, out_features=256, bias=True)
  )
Distribution:
  q(y|z) (Categorical)
Network architecture:
  Classifier(
    (fc1): Linear(in_features=256, out_features=10, bias=True)
  )


In [6]:
kl = KullbackLeibler(p, prior)
exp = StochasticExpectationLoss(p, q)
loss = (-exp + 1e-3 * kl).mean()
print(loss)

mean(-(E_p(z|x)[log q(y|z)]) + KL[p(z|x)||r(z)] * 0.001)


In [7]:
model = Model(loss, distributions=[p, q],
              optimizer=optim.Adam, optimizer_params={"lr":1e-4, "betas":(0.5, 0.999)})
print(model)

# Learning Rate Schedule
scheduler = StepLR(model.optimizer, step_size=2, gamma=0.97)

Distributions (for training): 
  p(z|x), q(y|z) 
Loss function: 
  mean(-(E_p(z|x)[log q(y|z)]) + KL[p(z|x)||r(z)] * 0.001) 
Optimizer: 
  Adam (
  Parameter Group 0
      amsgrad: False
      betas: (0.5, 0.999)
      eps: 1e-08
      initial_lr: 0.0001
      lr: 0.0001
      weight_decay: 0
  )


In [8]:
def train(epoch):
    scheduler.step()
    train_loss = 0
    for x, y in tqdm(train_loader):
        x = x.to(device)
        y = torch.eye(10)[y].to(device)
        loss = model.train({"x": x, "y": y})
        train_loss += loss
        
    train_loss = train_loss * train_loader.batch_size / len(train_loader.dataset)
    print('Epoch: {} Train loss: {:.4f}'.format(epoch, train_loss))
    
    return train_loss

In [9]:
def test(epoch):
    test_loss = 0
    correct = 0
    total = 0    
    for x, y in tqdm(test_loader):
        x = x.to(device)
        y = torch.eye(10)[y].to(device)        
        loss = model.test({"x": x, "y": y})
        test_loss += loss
        
        z = p.sample_mean({"x": x})
        pred_y = q.sample_mean({"z": z})
        total += y.size(0)
        correct += (pred_y.argmax(dim=1) == y.argmax(dim=1)).sum().item()      

    test_loss = test_loss * test_loader.batch_size / len(test_loader.dataset)
    test_accuracy = 100 * correct / total
    print('Test loss: {:.4f}, Test accuracy: {:.4f}'.format(test_loss, test_accuracy))
    return test_loss, test_accuracy

In [10]:
writer = SummaryWriter()

for epoch in range(1, epochs + 1):
    train_loss = train(epoch)
    test_loss, test_accuracy = test(epoch)
    
    writer.add_scalar('train_loss', train_loss.item(), epoch)
    writer.add_scalar('test_loss', test_loss.item(), epoch)
    writer.add_scalar('test_accuracy', test_accuracy, epoch)        
    
writer.close()

  0%|          | 0/100 [00:00<?, ?it/s]

Epoch: 1 Train loss: 0.6363


  0%|          | 0/600 [00:00<?, ?it/s]

Test loss: 0.3450, Test accuracy: 93.3800


  0%|          | 0/100 [00:00<?, ?it/s]

...

Test loss: 0.0975, Test accuracy: 98.7500


  0%|          | 0/100 [00:00<?, ?it/s]

Epoch: 200 Train loss: 0.0175


100%|██████████| 100/100 [00:01<00:00, 80.64it/s]

Test loss: 0.0971, Test accuracy: 98.7400
